Номер группы, ФИО:
М8О-202М-21, Базыльникова Екатерина Родионовна

In [130]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

Считываем данные из файла 'ph-data.csv'. Первые три столбца будут выступать в качестве матрицы признаков объектов - создать из них новый объект X. Целевая переменная хранится в столбце с названием label - из нее создать новый объект y. Заменить все значения y, меньшие 6 (кислотная среда) на -1, большие 9 (щелочная среда) на 1, 6-9 на 0.

In [131]:
from google.colab import files
uploaded = files.upload()
ph_data=pd.read_csv('ph-data.csv')

Saving ph-data.csv to ph-data.csv


Разбить данные на тренировочные и тестовые в соотношении 7/3. Зафиксировать random_state=5

In [133]:
# изменить порядок BGR->RGB
ph_df = ph_data[ph_data.columns.tolist()[::-1]]
# установить классы -1, 0, 1
ph_df.label[ph_df.label<6] = -1
ph_df.label[(5<ph_df.label) & (ph_df.label<10)] = 0
ph_df.label[ph_df.label>9] = 1

X = ph_df.drop('label', axis=1)
y = ph_df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=5)
print(y_train[:10])
print(len(y_train))

517    1
312   -1
349    0
189    1
116    1
208    1
178    1
417   -1
495    0
386    1
Name: label, dtype: int64
457


Создать и обучить поочередно классификаторы KNeighborsClassifier, GaussianNB, DecisionTreeClassifier. Обучить их на тренировочной выборке, оценить точность на проверочной. Попробовать изменить параметры в конструкторе n_neighbors у ближайших соседей, max_depth у дерева. Посмотреть как изменится точность предсказаний (и изменится ли).

In [134]:
clr = KNeighborsClassifier(n_neighbors=5)
clr.fit(X_train, y_train)
y_pred = clr.predict(X_test)
print('knn acc (5): ', accuracy_score(y_test, y_pred))
# точность не совсем зависит от количества соседей, при 7 соседях 0.92, при 5 0.94, при 20 0.93

clr = GaussianNB()
clr.fit(X_train, y_train)
y_pred = clr.predict(X_test)
#print(y_pred)
print('gaussian nb: ', accuracy_score(y_test, y_pred))

clr = DecisionTreeClassifier(max_depth=3)
clr.fit(X_train, y_train)
y_pred = clr.predict(X_test)
print('decision tree classifier (3): ', accuracy_score(y_test, y_pred))
#аналогично алгоритму KNN, точность достигает пика в каком-то месте, а потом снова ухудшается

knn acc (5):  0.9438775510204082
gaussian nb:  0.9285714285714286
decision tree classifier (3):  0.9540816326530612


Выбрать из просмотренных классификаторов тот классификатор и тот параметр конструктора, который показал большую точность. Обучить его на полной выборке X, y.

In [141]:
#выбираю дерево решений
#X_train_new, X_test, y_train_new, y_test = train_test_split(X, y, test_size=None, random_state=5)
clr = DecisionTreeClassifier(max_depth=3)
clr.fit(X.values, y)

DecisionTreeClassifier(max_depth=3)

Проверить, какой результат вернет классификатор для следующих объектов, заданных в формате RGB:

<br>
<code style="background:rgb(255, 38, 0);color:white">[255, 38, 0]</code>
<br>
<code style="background:rgb(255, 124, 0);color:white">[255, 124, 0]</code>
<br>
<code style="background:rgb(141, 250, 0);color:black">[141, 250, 0]</code>
<br>
<code style="background:rgb(179, 68, 198);color:white">[179, 68, 198]</code>
<br>
<code style="background:rgb(111, 43, 142);color:white">[111, 43, 142]</code>

In [142]:
X_new = [[255, 38, 0],[255, 124, 0],[141, 250, 0],[179, 68, 198],[111, 43, 142]]
#должно быть [-1, -1, 0, 1, 1]

In [143]:
y_pred = clr.predict(X_new)
print(y_pred)

[-1 -1 -1  1  1]


In [144]:
from joblib import dump
dump(clr,'baz_model.mdl')

['baz_model.mdl']

In [109]:
yyyy = clr.predict([[141,250,0]])
print(yyyy)

[-1]


In [145]:
from joblib import load
model = load('baz_model.mdl')
y_aaa = model.predict([[250,38,0]])
print(y_aaa[0])

-1
